<a href="https://colab.research.google.com/github/createnewdemo/pytorch/blob/main/%E7%AE%80%E5%8D%95%E7%9A%84%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#导入库
import  torch
import torch.nn as nn

In [ ]:
#开始构建神经网络
class Classifier(nn.Module):


  def __init__(self):
    #初始化Pytorch父类
    super().__init__()

    #PyTorch.nn模块会为我们设置分类器

    #定义神经网网络 -----------------------模块加正向的信息传导  还要定义如何计算误差以及用误差更新网络的可学习参数
    self.model = nn.Sequential(
        nn.Linear(784,200),             #是从784个节点到200个节点的全连接映射   这个模块包含节点之间连接的权重  在训练时会被更新
        nn.Sigmoid(),              # 将S型逻辑激活函数应用于前一个模块的输出  也就是本例中200个节点的输出
        nn.Linear(200,10),   #将200个节点映射到10个节点的全连接映射   它包含中间隐藏层与输出层10个节点之间所有连接的权重
        nn.Sigmoid()   # 再将S型逻辑激活函数应用于10个节点的输出  其结果就是网络的最终输出
    )

    #创建损失函数
    self.loss_function = nn.MSELoss

    #创建优化器，使用简单的梯度下降------随机梯度下降
    # 把所有可学习参数都传递给SGD优化器  这些参数可以通过self.parameters() 访问
    self.optimiser = torch.optim.SGD(self.parameters(),lr=0.01)
    pass

  #PyTorch 假定通过一个forward()方法 向网络传递信息
  def forward(self,input):
    #直接运行模型
    return self.model(input)

  def train(self,input,targets):
    #计算网络的输出值
    outputs = self.forward(input)

    #计算损失值
    loss = self.loss_function(outputs,targets)

    #梯度归0  反向传播  并更新权重
    self.optimiser.zero_grad()
    loss.backward()
    self.optimise.step()
    
    #记录训练进展的计数器和列表

    self.counter = 0
    self.progress = []

    #每隔10个训练样本增加一次计数器的值   并将损失值添加进列表的末尾
    self.counter += 1

    if (self.counter % 10 == 0):
      self.progress.append(loss.item())  # loss.item()   方便展开一个单值张量  获取里面的数字
      pass
    
    if (self.counter % 10000 == 0):
      print("counter=",self.counter)
      pass

    #损失值绘图
  def plot_progress(self):
    df = pandas.DataFrame(self.progress,columns=['loss'])
    df.plot(ylim=(0,1.0),figsize=(16,8),alpha=0.1,marker='.')
    pass


from torch.utils.data import Dataset

class MnistDataset(Dataset):
  
  def __init__(self,csv_file):

  






